In [1]:
import json
import re
import time
from statistics import mean, stdev
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
!pip install langchain langchain-community

Defaulting to user installation because normal site-packages is not writeable


In [17]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage

In [15]:
import json
import matplotlib.pyplot as plt
#from chatollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage

# ---- Config ---- #
STUDENT_MODEL = "llama3.1:8b"
TEACHER_MODEL = "llama3.1:70b"#

SCORE_WEIGHTS = {
    "accuracy": 0.5,
    "completeness": 0.2,
    "relevance": 0.15,
    "clarity": 0.15
}
INTERVENTION_THRESHOLD = 0.8
SCORE_MAX = 3

# ---- Penalty Function ---- #
def should_teacher_intervene(scores, threshold=0.8):
    """
    Determine if the teacher LLM should intervene based on scores.
    
    Args:
        scores (dict): Dictionary with keys 'accuracy', 'completeness', 'relevance', 'clarity',
                       each with integer score 1 to 3.
        threshold (float): Penalty threshold for intervention.
        
    Returns:
        bool: True if teacher should intervene, False otherwise.
    """
    a = scores['accuracy']
    c = scores['completeness']
    r = scores['relevance']
    l = scores['clarity']

    # Compute weighted penalty
    penalty = 0.5 * (3 - a) + 0.2 * (3 - c) + 0.15 * (3 - r) + 0.15 * (3 - l)

    # Suspicious patterns
    suspicious = (
        (a == 1 and l == 3) or                          # Fluent but inaccurate
        (a >= 2 and r == 1) or                          # Correct but off-topic
        (a == 2 and r == 2 and l == 2) or               # Ambiguous scores
        (a + r + l <= 5) or                             # Generally low scores
        (abs(a - r) >= 2) or                            # Large inconsistency
        (a == 3 and (r <= 1 or l <= 1))                 # Overconfident accuracy
    )

    # Decide intervention
    return penalty > threshold or suspicious




In [16]:
#------- Original prompt used by Student LLM to produce scores. Student LLM will use this prompt again after receiving teacher's feedback---#
def call_llm_LangChain_correct_v2(prompt, temperature=0.0):
    llm = ChatOllama(
        model="llama3.1:8b",
        temperature=temperature
    )

    system_prompt = """

You are a large language model acting as a judge for assessing the performance of a Teaching Assistant (TA) in an introductory Python programming course.

The TA is an LLM that answers student questions about Python code. Your job is to evaluate the quality of the TA's answer.

You will receive:
- A Python code snippet
- A student question about that code
- A reference (correct) answer
- A TA LLM-generated answer (called the prediction)

Your task is to evaluate how well the TA's prediction answers the student's question, using the following four dimensions. For each, provide:
- An integer score from 1 to 3



### Accuracy
Compare the prediction with the reference to assess factual correctness and understanding of the code’s behavior and intent.
You must judge whether the prediction reflects accurate behavior and matches core facts from the reference. 
You need to consider semantic meaning of code comprehension: understanding the structure, functionality, and intent behind the code.\n"

Score meanings:
- 1: Completely incorrect or irrelevant; does not address the reference answer.
- 2: Partially correct; some key facts are accurate, but major details are wrong or missing.
- 3: Fully correct; matches the reference answer in meaning and factual content.

### Completeness
Check if the prediction covers all important parts of the reference answer, including key concepts or conditions.

Score meanings:
- 1: Omits most key information or contains only a tiny fragment of relevant content.
- 2: Covers some elements but misses important parts.
- 3: Fully covers all essential information from the reference.

### Relevance
Assess whether the prediction directly addresses the question and stays on-topic.

Score meanings:
- 1: Completely irrelevant or mostly unrelated.
- 2: Partially related but misses the main point.
- 3: Fully focused and directly answers the question.

### Clarity
Evaluate how clearly and logically the prediction is expressed, ensuring it is easy to understand.

Score meanings:
- 1: Confusing, vague, or incoherent.
- 2: Understandable but awkwardly phrased or slightly unclear.
- 3: Clear, concise, and easy to follow.


Example:

Code:
```python
def count_even(nums):
    total = 0
    for x in nums:
        if x % 2 == 0:
            total += 1
    return total
Question: What does this function return when given a list of integers?
Reference Answer: It returns the count of even numbers in the list.
Prediction: It returns the count of odd numbers in the list.

Evaluation Output:
{

"accuracy": { "score": 1 },
"completeness": { "score": 1 },
"relevance": { "score": 2 },
"clarity": { "score": 3 }

}

Final Instructions:
For the given input (code, question, reference answer, and prediction), evaluate the prediction on the four metrics defined above.
Base your evaluation strictly on the content provided. Do not hallucinate missing information. Be consistent and objective.
Do not include reasoning or explanations.

Respond only with a JSON object in the exact format:
{
"accuracy": { "score": 1-3},
"completeness": {"score": 1-3},
"relevance": {"score": 1-3},
"clarity": {"score": 1-3}
}
"""

  

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=prompt)
    ]

    response = llm.invoke(messages)
    return response.content


In [18]:
# ---- Step 3: Student provides reasoning after teacher prompts ---- #
def generate_reasoning_after_teacher_prompt(code, question, reference, prediction, scores):
    reasonings = {}
    for dim, score in scores.items():
        prompt = f"""
The teacher is requesting your reasoning for why you scored {score}/3 on {dim.upper()}.

Explain briefly (1–2 sentences) based on the predicted answer vs. the reference.
"""
        full_input = f"""
Code:
```python
{code}
Question: {question}

Reference Answer:
{reference}

Predicted Answer:
{prediction}
"""
        llm = ChatOllama(model=STUDENT_MODEL, temperature=0.3)
        messages = [SystemMessage(content=prompt), HumanMessage(content=full_input)]
        response = llm.invoke(messages)
        reasonings[dim] = response.content.strip()
    return reasonings

In [19]:
# ---- Step 4: Teacher critiques ---- #
def teacher_critiques_student(code, question, reference, prediction, reasonings, scores):
    critiques = {}
    for dim, reasoning in reasonings.items():
        score = scores.get(dim, SCORE_MAX)
        if score < SCORE_MAX:
            system_prompt = f"""
You are a 70B teacher model. The student scored {score}/3 for {dim.upper()} and gave this reasoning:
\"{reasoning}\"


You will receive:
- Python code snippet
- Student's question
- Reference (correct) answer
- TA's predicted answer
- Scores assigned by the student LLM-as-judge
- Reasoning 

Your task:
- Examine the TA's predicted answer in context of the code, question, and reference.
- For any dimension (Accuracy, Completeness, Relevance, Clarity) where the score is less than 3,
  provide clear, concise feedback (2–4 sentences) explaining what could be improved.
- If a dimension has no issues, do not include it in your response.
- Critique the reasoning in 3–5 sentences. Point out flaws and suggest improvement.

Respond ONLY with a JSON object where keys are the dimension names (lowercase)
and values are the feedback strings.

Example output:
{{
  "accuracy": "The prediction misrepresents the function’s return value.",
  "clarity": "The explanation lacks structure and is hard to follow."
}}

Rubric:

### Accuracy
- 1: Completely incorrect or irrelevant.
- 2: Partially correct but with major mistakes or omissions.
- 3: Fully correct and matches the reference.

### Completeness
- 1: Omits most key information.
- 2: Covers some but misses important parts.
- 3: Fully covers all essential information.

### Relevance
- 1: Irrelevant or mostly unrelated.
- 2: Partially related but misses main point.
- 3: Fully focused and directly addresses the question.

### Clarity
- 1: Confusing, vague, or incoherent.
- 2: Understandable but awkwardly phrased or unclear.
- 3: Clear, concise, and easy to understand.
"""
            user_prompt = (
                "Code:\n"
                "```python\n"
                f"{code}\n"
                "```\n"
                f"Question: {question}\n\n"
                f"Reference Answer:\n{reference}\n\n"
                f"Predicted Answer:\n{prediction}"
            )

            llm = ChatOllama(model=TEACHER_MODEL, temperature=0.0)
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=user_prompt)
            ]
            response = llm.invoke(messages)
            critiques[dim] = response.content.strip()
    return critiques


In [20]:
# ---- Step 5: Student Judge LLM reflects an revises  ---- #
def student_reflect_and_revise(code, question, reference, prediction, old_score, critiques):
    print("\n Student reflecting on teacher feedback...\n")

    critique_text = "\n".join(
        f"{dim.upper()} Feedback: {critique.strip()}" for dim, critique in critiques.items()
    )

    prompt = f"""
Code:
```python
{code}
Question:
{question}

Reference Answer:
{reference}

TA's Predicted Answer:
{prediction}

Teacher's Feedback:
{critique_text}

Old prediction:
{old_score}

You have to update the old prediction by considering Teacher's Feedback. Please re-evaluate the TA's answer using:

Accuracy

Completeness

Relevance

Clarity

Only return JSON:
{{
"accuracy": {{ "score": 1-3 }},
"completeness": {{ "score": 1-3 }},
"relevance": {{ "score": 1-3 }},
"clarity": {{ "score": 1-3 }}
}}
"""

    try:
        response = call_llm_LangChain_correct_v2(prompt, temperature=0.0)
        revised_scores = json.loads(response)
        print(" Revised Scores:\n", revised_scores)
    except Exception as e:
        print(" Failed to parse revised scores:", str(e))
        revised_scores = None

    return revised_scores


In [21]:
# ---- Step 1: Student Judge LLM provides scores  ---- #

example = {"id": "q132",
   "code": "from cs1robots import* load_world(\"worlds/harvest1.wld\") #load world(\"worlds/harvest3.wld\") #load world(\"worlds/harvest4.wld\") hubo=Robot() hubo.set_trace('blue') def harvest(): if hubo.on_beeper(): hubo.pick_beeper() else: hubo.move() def right(): for i in range(3): hubo.turn_left() def right_up(): if hubo.on_beeper(): hubo.pick_beeper() hubo.turn_left() hubo.move() hubo.turn_left() def left_up(): right() hubo.move() right() def loop(): while hubo.front_is_clear(): harvest() right_up() while hubo.front_is_clear(): harvest() for i in range(2): loop() left_up() loop()",
    "question": "In task4, do I have to pass only the beeper?",
    "reference": "No, you don't need to, just pick up all the beepers!",
    "prediction": "\nHello! I understand that you are asking about a specific task related to programming in Python. However, your question is quite broad and doesn't provide enough context for me to fully understand what you need help with. Could you please provide more details or clarify your question so that I can better assist you?",
  }
studentLLM_score = {"accuracy": 1, "completeness": 2, "relevance": 1, "clarity": 3}




In [22]:
# ---- Step 2: Decide whether the teacher LLM will intervene  ---- #
do_intervene = should_teacher_intervene(studentLLM_score, threshold=0.8)



In [13]:
print(do_intervene)

True


In [ ]:
teacher_feedbacks =0
if (do_intervene):
    #example["code"], example["question"], example["reference"], example["prediction"], studentLLM_score
    reasoning_std = generate_reasoning_after_teacher_prompt(example["code"], example["question"], 
                                    example["reference"], example["prediction"], studentLLM_score)
    print("reasoning ", reasoning_std)
    critiques= teacher_critiques_student(example["code"], example["question"], example["reference"], example["prediction"], reasoning_std, studentLLM_score)
    print("critices ", critiques)
    revised_scores = student_reflect_and_revise(example["code"], example["question"], example["reference"], example["prediction"], studentLLM_score, critiques)
    
    
    
    

reasoning  {'accuracy': "Here's a brief explanation of why the predicted answer scored 1/3 on ACCURACY:\n\nThe predicted answer was a generic response that didn't address the specific task at hand, whereas the reference answer provided a clear and concise solution to the problem. The predicted answer failed to understand the context of the question and provide a relevant response, resulting in low accuracy.", 'completeness': "Here's a brief explanation for scoring 2/3 on COMPLETENESS:\n\nI scored low on completeness because my predicted answer failed to directly address the specific task of Task4, instead providing a generic response about understanding the context. A more complete response would have explicitly stated that the robot is designed to pick up all beepers in its path, not just pass them by.", 'relevance': "Here's a brief explanation of why the predicted answer scored 1/3 on RELEVANCE:\n\nThe predicted answer is off-topic and doesn't address the specific task in question (t

In [ ]:
#results = run_multi_turn_pipeline(example)